In [1]:
import pandas as pd
from datetime import datetime, timedelta

def first_friday(year, month):
    d = datetime(year, month, 1)
    days_until_friday = (4 - d.weekday()) % 7
    return d + timedelta(days=days_until_friday)

# anniversaries.csv 읽기
anniversaries = pd.read_csv("anniversaries.csv", encoding="euc-kr")

In [2]:
# 현재 연도 구하기
current_year = datetime.now().year

# 4월 첫째 금요일 날짜 계산
first_friday_date = first_friday(current_year, 4).strftime("%m-%d")  # ex) '04-04'

# '기념일' 또는 '설명' 컬럼에서 '4월 첫째 금요일' 찾기 (예: '예비군의 날' 행)
# 여기서는 '날짜' 컬럼이 '4-첫째-금요일' 이런 식이라 가정

mask = anniversaries['date'].str.contains("4-1st-fri", case=False, na=False)

# 해당 행의 날짜 컬럼 업데이트
anniversaries.loc[mask, 'date'] = first_friday_date

# 변경된 데이터프레임 저장 (덮어쓰기)
anniversaries.to_csv("anniversaries.csv", index=False, encoding="euc-kr")

print(f"4월 첫째 금요일 날짜를 {first_friday_date}로 업데이트했습니다.")

4월 첫째 금요일 날짜를 04-04로 업데이트했습니다.


In [3]:
import pandas as pd
from konlpy.tag import Okt
import json

# 1️⃣ CSV 불러오기
df = pd.read_csv("anniversaries.csv", encoding="euc-kr")

In [4]:
# 2️⃣ 형태소 분석기 초기화
okt = Okt()

In [5]:
# 3️⃣ 불용어 리스트 정의
stopwords = ["정신", "행사", "위해", "관련", "기념", "계승", "발전", "하는", "고취", "함양", "진흥", "촉진", "새롭게", "임무", "기념일", "대규모", "중심", "정신과", "길이"]

# 4️⃣ 키워드 추출 함수
def extract_keywords(text):
    nouns = okt.nouns(text)  # 설명문에서 명사만 추출
    filtered = [word for word in nouns if word not in stopwords and len(word) > 1]
    return list(set(filtered))  # 중복 제거

In [6]:
# 5️⃣ 각 기념일 설명문에서 키워드 추출
df['keywords'] = df['desc'].apply(extract_keywords)

# 6️⃣ 결과 출력
print(df[['name', 'date', 'keywords']])

# 7️⃣ JSON으로 저장
df[['name', 'date', 'keywords']].to_json("anniversary_keywords.json", orient="records", force_ascii=False, indent=2)

# ✅ CSV로도 저장
df.to_csv("anniversary_keywords.csv", encoding="utf-8-sig", index=False)

             name     date                                       keywords
0            3·1절  03월 01일  [독립, 단결, 선열, 위업, 자존, 국권, 회복, 애국심, 취함, 민족, 기리]
1         순국선열의 날  11월 17일                        [학생, 자율, 독립운동, 애국심, 역량]
2      학생독립운동 기념일  11월 03일                        [학생, 자율, 독립운동, 애국심, 역량]
3   5·18민주화운동 기념일  05월 18일                                  [광주, 민주화, 운동]
4    6·10민주항쟁 기념일  06월 10일                                       [항쟁, 민주]
5         6·25전쟁일  06월 25일                           [국민, 안보, 전쟁, 상기, 의식]
6     충무공 이순신 탄신일  04월 28일                                 [충무공, 충의, 이순신]
7    6·10만세운동 기념일  06월 10일         [독립, 강제, 희생, 순국선열, 지배, 항거, 병합, 식민, 일제]
8           국군의 날  10월 01일                 [국내외, 위용, 국군, 사기, 장병, 과시, 전투력]
9           의병의 날  06월 01일                       [희생, 나라, 사랑, 국민, 의병, 기리]
10         예비군의 날  04월 04일                      [참가, 방위, 모든, 국가, 다짐, 예비군]
11           어린이날  05월 05일                                      [슬기, 어린이]
12        우주항공의 날  05월 27일            

In [57]:
import pandas as pd
from datetime import datetime

# 📦 데이터 불러오기
df = pd.read_csv("kmdb_csv.csv", encoding="euc-kr")
df['키워드'] = df['키워드'].fillna('').astype(str)
df['줄거리'] = df['줄거리'].fillna('').astype(str)
df['대표개봉일'] = df['대표개봉일'].fillna('').astype(str).str.strip()

anniversaries = pd.read_csv("anniversary_keywords.csv", encoding="utf-8")

# 🎯 오늘 날짜 확인
today = '05월 18일'  # datetime.now().strftime("%m월 %d일")

# 🎯 오늘 해당하는 기념일 찾기
row = anniversaries[anniversaries['date'] == today]

if row.empty:
    print("📅 오늘은 기념일이 아닙니다.")
else:
    print(f"📅 오늘은 '{row.iloc[0]['date']}'입니다.")

    # 1️⃣ 키워드 파싱
    keywords_raw = row.iloc[0]['keywords']
    try:
        keywords = eval(keywords_raw) if isinstance(keywords_raw, str) else []
    except:
        keywords = [k.strip() for k in keywords_raw.split(',') if k.strip()]
    
    print(f"🎯 검색 키워드: {keywords}")

    # 2️⃣ 영화 필터링 함수
    def match_keywords(movie_keywords_str, keywords, min_match=2):
        movie_keywords = [k.strip() for k in movie_keywords_str.split(',') if k.strip()]
        match_count = sum(1 for kw in keywords if any(kw in mk or mk in kw for mk in movie_keywords))
        return match_count >= min_match

    # 3️⃣ 키워드 매칭
    df['추천여부'] = df['키워드'].apply(lambda x: match_keywords(x, keywords, min_match=2))
    filtered = df[df['추천여부']].copy()  # ← 여기를 'filtered'로 맞춰야 함

    if filtered.empty:
        print("😢 해당 키워드에 맞는 영화가 없습니다.")
    else:
        # 4️⃣ 개봉일 파싱 함수
        def parse_release_date(date_str):
            try:
                clean = date_str.replace('.0', '')[:8]  # '20170727.0' → '20170727'
                return datetime.strptime(clean, '%Y%m%d')
            except:
                try:
                    return datetime.strptime(date_str.strip()[:4], "%Y")  # 연도만 있는 경우
                except:
                    return None  # 파싱 실패

        filtered['개봉일파싱'] = filtered['대표개봉일'].apply(parse_release_date)
        filtered = filtered.sort_values(by='개봉일파싱', ascending=False)

        # 5️⃣ 출력용 날짜 포맷 (보기 좋게)
        def format_date_for_display(date_str):
            try:
                d = parse_release_date(date_str)
                return d.strftime('%Y-%m-%d') if d else '미상'
            except:
                return '미상'
        
        filtered['표시용개봉일'] = filtered['대표개봉일'].apply(format_date_for_display)

        # 6️⃣ 결과 출력
        print("🆕 기념일 추천 영화 (최신순):")
        print(filtered[['표시용개봉일', '영화등록번호NO', '줄거리']].head(20))

C:\Users\김찬은\AppData\Local\Temp\ipykernel_8964\2566684549.py:5: DtypeWarning: Columns (16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("kmdb_csv.csv", encoding="euc-kr")


📅 오늘은 '05월 18일'입니다.
🎯 검색 키워드: ['광주', '민주화', '운동']
🆕 기념일 추천 영화 (최신순):
       표시용개봉일        영화등록번호NO  \
K  2022-05-18       쏴!쏴!쏴!쏴!탕   
K  2021-05-12        아들의 이름으로   
A  2020-07-16  광주비디오: 사라진 4시간   
A  2019-05-23              김군   
A  2018-10-31         1991, 봄   
A  2018-05-17   5.18 힌츠페터 스토리   
K  2018-05-16       임을 위한 행진곡   
K  2017-12-27            1987   
K  2017-08-02           택시운전사   
K  2017-07-27            포크레인   
K  2012-12-06            네모난원   
K  2012-11-29             26년   
K  2012-11-22         남영동1985   
K  2012-10-25   강철대오: 구국의 철가방   
A  2011-05-12             오월愛   
K  2008-01-31      슈퍼맨이었던 사나이   
K  2007-11-15            스카우트   
K  2007-07-25          화려한 휴가   
K  2007-01-04          오래된 정원   
K  2000-01-01            박하사탕   

                                                 줄거리  
K  각자 5.18 당시 피해자와 가해자의 기억으로 살아가고 있는 세 남녀의 엇갈린 운명...  
K  주인공 채근은 우울증에 걸린 대리운전 기사다. 그는 미국에 유학간 아들과 종종 통화...  
A  인생을 송두리째 뒤흔든 영화가 ‘광주비디오’인 사람들이 있다.\r\n그들은 80년 ...  
A  영화 <김군>은 1980년